In [1]:
import mxnet as mx
from mxnet import init, gluon, nd, autograd, image
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import h5py
import os
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

ctx = mx.gpu()

In [2]:
def transform1(data, label):
    data = data.astype('float32') / 255
    pre_224 = [image.ForceResizeAug((224,224)), image.ColorNormalizeAug(
        mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225]))]
    for pre in pre_224:
        data = pre(data)
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

def transform2(data, label):
    data = data.astype('float32') / 255
    pre_299 = [image.ForceResizeAug((299,299)), image.ColorNormalizeAug(
        mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225]))]
    for pre in pre_299:
        data = pre(data)
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

def save_features(model_name, data_iter, ignore=False):
    if os.path.exists('features_test_%s.nd' % model_name):
        return
    net = models.get_model(model_name, pretrained=True, ctx=ctx)
    features = []
    for data, _ in tqdm(data_iter):
        feature = net.features(data.as_in_context(mx.gpu()))
        feature = gluon.nn.Flatten()(feature)
        features.append(feature.as_in_context(mx.cpu()))
        nd.waitall()
    features = nd.concat(*features, dim=0)
    nd.save('features_test_%s.nd' % (model_name), features)

In [3]:
df = pd.read_csv('labels.csv')
synset = sorted(set(df['breed']))
n = len(df)
lis_y = []
for i, (fname, breed) in tqdm(df.iterrows(), total=n):
    lis_y.append(synset.index(breed))
    nd.waitall()
y = nd.array(lis_y).reshape((n,1))

imgs_224_test = vision.ImageFolderDataset('for_test', transform=transform1)
imgs_299_test = vision.ImageFolderDataset('for_test', transform=transform2)

data_test_iter_224 = gluon.data.DataLoader(gluon.data.ArrayDataset(imgs_224_test), batch_size=128)
data_test_iter_299 = gluon.data.DataLoader(gluon.data.ArrayDataset(imgs_299_test), batch_size=128)

100%|██████████| 10222/10222 [00:00<00:00, 21065.56it/s]


In [4]:
model_1 = 'inceptionv3'

save_features(model_1, data_test_iter_299)

100%|██████████| 81/81 [01:55<00:00,  1.43s/it]


In [ ]:
model_2 = 'resnet152_v1'

save_features(model_2, data_test_iter_224)